In [1]:
vowels = ['а', 'е', 'є', 'и', 'і', 'ї', 'о', 'у', 'ю', 'я']
consonants = ['б', 'в', 'г', 'ґ', 'д', 'й', 'ж', 'з', 'к', 'л', 'м', 'н', 'п', 'р', 'с', 'т', 'ф', 'х', 'ц', 'ч', 'ш', 'щ']
soften = 'ь'
preSoften = ['д', 'т', 'з', 'с', 'ц', 'л', 'н']
#For now, there will be no apostrophe

#The formula for the number of skip-grams will be wordLetterNumber^2 * magicX
wordLetterNumber = 10
magicX = 0.1
#Probability of next letter to be a vowel / consonant
vowelProb = 0.5
consonantProb = 0.5
softenProb = 0.5

In [2]:
#Generate the random-formed word from vowels and consonants
#This is a COMPLETE RANDOM selector. Markov chain or something similar can give better letter prediction
import random

def GenerateWord(wordLetterNumber, vowelProb, consonantProb, softenProb):
  word = ""
  prevLetter = ''
  curLetter = ''
  for i in range(0, wordLetterNumber):
    #The probability of the next letter to be a vowel. If it's greater than the vowelProb, the letter is a consonant
    probability = random.random()
    if probability <= vowelProb:
      word = word + random.choice(vowels)
      if i == 0:
        prevLetter = 'v'
        curLetter = 'v'
      else:
        prevLetter = curLetter
        curLetter = 'v'
    else:
      #If we got a consonant, we may, with some prob, get a softened one
      randConsonant = random.choice(consonants)
      softProbability = random.random()
      if softProbability <= softenProb and randConsonant in preSoften:
        word = word + randConsonant + soften
      else:
        word = word + randConsonant
      if i == 0:
        prevLetter = 'c'
        curLetter = 'c'
      else:
        prevLetter = curLetter
        curLetter = 'c'
    if prevLetter == curLetter and i != 0:
      if prevLetter == 'v':
        probToShare = vowelProb / 2
        vowelProb = vowelProb - probToShare
        consonantProb = consonantProb + probToShare
      elif prevLetter == 'c':
        probToShare = consonantProb / 2
        vowelProb = vowelProb + probToShare
        consonantProb = consonantProb - probToShare
    print("W: " + word + " NextV: " + str(vowelProb) + " NextC: " + str(consonantProb))
    if len(word) >= 10:
      break
  return word

print(GenerateWord(wordLetterNumber, vowelProb, consonantProb, softenProb))

W: е NextV: 0.5 NextC: 0.5
W: ед NextV: 0.5 NextC: 0.5
W: еді NextV: 0.5 NextC: 0.5
W: едіб NextV: 0.5 NextC: 0.5
W: едібо NextV: 0.5 NextC: 0.5
W: едібоу NextV: 0.25 NextC: 0.75
W: едібоум NextV: 0.25 NextC: 0.75
W: едібоумі NextV: 0.25 NextC: 0.75
W: едібоуміх NextV: 0.25 NextC: 0.75
W: едібоуміхж NextV: 0.625 NextC: 0.375
едібоуміхж


In [3]:
#Connect to Google Drive
#https://github.com/lang-uk/ukrainian-word-stress-dictionary/
#https://pypi.org/project/markov-word-generator/

from google.colab import drive
drive.mount('/content/gdrive')

#Install the word generation library (core algorithm is the Markov chain)
!pip install markov-word-generator
!pip install unidecode
import unidecode
from markov_word_generator import MarkovWordGenerator

#And generate the word similar to the real Ukrainian word
#The dictionary is my custom file found by the first link, cleared from stress symbols and duplicates
generator = MarkovWordGenerator(
	markov_length=4,
	dictionary_filename='/content/gdrive/My Drive/Kaggle/ukr-word-dict.txt'
)
print(generator.generate_word())

Mounted at /content/gdrive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.6 MB/s eta 0:00:00
хромочками


In [5]:
words = []
wordNumber = 10

#Generate some words for further analysis
while len(words) < wordNumber:
  word = generator.generate_word()
  if len(word) > 5:
    words.append(generator.generate_word())

for word in words:
  print(word)

дере
переряджуймо
спеціалозабудкоокоссорельнім
шубованим
декламним
видай
чкалачна
відеоцинко
мишаних
зашиваної


In [6]:
def skipgramFormer(word, index, substringLength):
  skipgram = word[:index]
  skipgram = skipgram + word[index + substringLength:]
  return skipgram

print(skipgramFormer("ігровий", 2, 3))

ігий


In [16]:
word = "урисушабив"

#Generating skipgrams
#Skipgram array
skipgrams = []
skipgramFstRemoved = []
skipgramSndRemoved = []
#Keep the lengths of all parts - useful when restoring the word
skipgramLengths = []

#Choose a random word to process
word = random.choice(words)
skipgramNumber = int(len(word)**2 * magicX)
print(skipgramNumber)
print(word)
for i in range (0, skipgramNumber):
  #There can be either one or two strips
  cutNumber = random.randint(1, 3)
  #The max substring length = length of word - 5
  maxSubstringLen = len(word) - 5
  if maxSubstringLen > 2:
    if cutNumber == 1:
      #For the only cut, we can strip any part of the word
      curSubstringLen = random.randrange(1, maxSubstringLen + 1)
      index = random.randrange(0, len(word) - curSubstringLen + 1)
      #Keep the lengths of all the word segments
      lengths = []
      lengths.append(index)
      lengths.append(curSubstringLen)
      lengths.append(len(word) - curSubstringLen - index)
      #And put everything to the corresponding arrays
      skipgramFstRemoved.append(word[index : (index + curSubstringLen)])
      skipgramSndRemoved.append("")
      skipgrams.append(skipgramFormer(word, index, curSubstringLen))
      skipgramLengths.append(lengths)
    else:
      #For two cuts, we need to strip two different parts of the word
      firstSubstrLen = random.randrange(1, maxSubstringLen - 1)
      secondSubstrLen = random.randrange(1, maxSubstringLen - firstSubstrLen + 1)
      #There must be at least one letter between the cuts
      lengthLeft = len(word) - firstSubstrLen - secondSubstrLen
      firstLeftOver = random.randrange(0, lengthLeft - 1)
      secondLeftOver = random.randrange(1, lengthLeft - firstLeftOver)
      thirdLeftOver = lengthLeft - firstLeftOver - secondLeftOver
      #When done, put the LeftOvers into the string - this is the Skipgram
      firstSkipgramPart = word[: firstLeftOver]
      secondSkipgramPart = word[firstLeftOver + firstSubstrLen : firstLeftOver + firstSubstrLen + secondLeftOver]
      thirdSkipgramPart = word[firstLeftOver + firstSubstrLen + secondLeftOver + secondSubstrLen :]
      skipgram = firstSkipgramPart + secondSkipgramPart + thirdSkipgramPart
      #And keep the removed parts, too
      firstRemoved = word[firstLeftOver : firstLeftOver + firstSubstrLen]
      secondRemoved = word[firstLeftOver + firstSubstrLen + secondLeftOver : firstLeftOver + firstSubstrLen + secondLeftOver + secondSubstrLen]
      #Keep the lengths
      lengths = []
      lengths.append(firstLeftOver)
      lengths.append(firstSubstrLen)
      lengths.append(secondLeftOver)
      lengths.append(secondSubstrLen)
      lengths.append(thirdLeftOver)
      #Put everything to correct arrays
      skipgramFstRemoved.append(firstRemoved)
      skipgramSndRemoved.append(secondRemoved)
      skipgrams.append(skipgram)
      skipgramLengths.append(lengths)

for i in range(0, len(skipgrams)):
  print("Sk: " + skipgrams[i] + " ; Fst: " + skipgramFstRemoved[i] + " ; Snd: " + skipgramSndRemoved[i])

6
чкалачна
Sk: чкачна ; Fst: ал ; Snd: 
Sk: чклчна ; Fst: а ; Snd: а
Sk: чкаача ; Fst: л ; Snd: н
Sk: чкачна ; Fst: ла ; Snd: 
Sk: чкачна ; Fst: ал ; Snd: 
Sk: чкалчна ; Fst: а ; Snd: 


In [12]:
#Read my dictionary file as a list of words
openedFile = open('/content/gdrive/My Drive/Kaggle/ukr-word-dict.txt', "r")
wordList = openedFile.read().split("\n")
openedFile.close()
print(wordList[:10])

['а', 'а вже ж', 'а все ж таки', 'а все-таки', 'а капела', 'а ні', 'а то', 'а то ж', 'а як же', 'а-конто']


In [13]:
#Find words similar to a part of the word
#https://pypi.org/project/python-Levenshtein/
!pip install python-Levenshtein
import Levenshtein
import difflib

def LevensteinSimilarity(word, dictionaryWordList, similarsNum):
  #Return -1 for empty word or for the word with nothing similar to find
  if len(word) == 0:
    return -1
  lenLimitDown = 0
  if len(word) > 2:
    lenLimitDown = len(word) - 1
  lenLimitUp = len(word) + 3
  #The list of words should be filtered by length
  tempWordList = list([x for x in dictionaryWordList if len(x) >= lenLimitDown and len(x) <= lenLimitUp])
  matches = difflib.get_close_matches(word, tempWordList, similarsNum)
  dictionary = {}
  for mat in matches:
    dictionary[mat] = -1
  #Weights are (insertion, deletion, substitution)
  for matche in matches:
    dist = Levenshtein.distance(word, matche, weights=(1,1,1))
    dictionary[matche] = dist
  newDict = dict(sorted(dictionary.items(), key=lambda x: (x[1],x[0])))
  #for key, value in newDict.items():
  #  print(str(key) + " " + str(value))
  return list(newDict.items())[0][1]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 11.9 MB/s eta 0:00:00


In [17]:
#Now check each part of the word - the skipgram and both cut-offs
#I'll set the max Levenshtein distance to 2 for now
maxLevenDist = 2
similarsNum = 15

goodWords = []
for i in range(0, len(skipgrams)):
  levenSkipgram = LevensteinSimilarity(skipgrams[i], wordList, similarsNum)
  levenFstRemoved = LevensteinSimilarity(skipgramFstRemoved[i], wordList, similarsNum)
  levenSndRemoved = LevensteinSimilarity(skipgramSndRemoved[i], wordList, similarsNum)
  print(str(levenSkipgram) + " " + str(levenFstRemoved) + " " + str(levenSndRemoved))
  #Now check if fits
  if levenSkipgram <= maxLevenDist and levenFstRemoved <= maxLevenDist and levenFstRemoved != -1 and levenSndRemoved <= maxLevenDist and levenSndRemoved != -1:
    #If all three fit, the word is good, and we need to put it back together
    restoreWord = ""
    lengths = skipgramLengths[i]
    restoreWord = restoreWord + skipgrams[i][:lengths[0]]
    restoreWord = restoreWord + skipgramFstRemoved[i]
    if skipgramSndRemoved[i] == "":
      restoreWord = restoreWord + skipgrams[i][lengths[0]:]
    else:
      restoreWord = restoreWord + skipgrams[i][lengths[0] : lengths[0] + lengths[2]]
      restoreWord = restoreWord + skipgramSndRemoved[i]
      restoreWord = restoreWord + skipgrams[i][lengths[0] + lengths[2] :]
    if restoreWord not in goodWords:
      goodWords.append(restoreWord)
  elif levenSkipgram <= maxLevenDist and levenFstRemoved <= maxLevenDist and levenFstRemoved != -1:
    #Second part doesn't fit, so we put together only the first and main part
    restoreWord = ""
    lengths = skipgramLengths[i]
    restoreWord = restoreWord + skipgrams[i][:lengths[0]]
    restoreWord = restoreWord + skipgramFstRemoved[i]
    restoreWord = restoreWord + skipgrams[i][lengths[0]:]
    if restoreWord not in goodWords:
      goodWords.append(restoreWord)
  elif levenSkipgram <= maxLevenDist and levenSndRemoved <= maxLevenDist and levenSndRemoved != -1:
    #First part doesn't fit, so we put together the main and second part
    restoreWord = ""
    lengths = skipgramLengths[i]
    restoreWord = restoreWord + skipgrams[i][: lengths[0] + lengths[2]]
    restoreWord = restoreWord + skipgramSndRemoved[i]
    restoreWord = restoreWord + skipgrams[i][lengths[0] + lengths[2] :]
    if restoreWord not in goodWords:
      goodWords.append(restoreWord)
  elif levenFstRemoved <= maxLevenDist and levenFstRemoved != -1 and levenSndRemoved <= maxLevenDist and levenSndRemoved != -1:
    #Main part doesn't fit, so we put together the first and second removed parts
    restoreWord = skipgramFstRemoved[i] + skipgramSndRemoved[i]
    if restoreWord not in goodWords:
      goodWords.append(restoreWord)
  elif levenSkipgram <= maxLevenDist:
    #Only the main skipgram fits
    if skipgrams[i] not in goodWords:
      goodWords.append(skipgrams[i])
  elif levenFstRemoved <= maxLevenDist and levenFstRemoved != -1:
    #Only the first removed part fits
    if skipgramFstRemoved[i] not in goodWords:
      goodWords.append(skipgramFstRemoved[i])
  elif levenSndRemoved <= maxLevenDist and levenSndRemoved != -1:
    #Only the second removed part fits
    if skipgramSndRemoved[i] not in goodWords:
      goodWords.append(skipgramSndRemoved[i])

for word in goodWords:
  print(word)

2 1 -1
2 0 0
2 1 1
2 1 -1
2 1 -1
2 0 -1
чкалачна


In [ ]:
#Test
word = "abcdefghijklmno"
lenWord = 15
maxSubstringLen = lenWord - 5
lengths = []
firstSubstrLen = random.randrange(0, maxSubstringLen - 1)
secondSubstrLen = random.randrange(1, maxSubstringLen - firstSubstrLen + 1)
lengthLeft = lenWord - firstSubstrLen - secondSubstrLen
firstLeftOver = random.randrange(0, lengthLeft - 1)
secondLeftOver = random.randrange(1, lengthLeft - firstLeftOver)
thirdLeftOver = lengthLeft - firstLeftOver - secondLeftOver
print(word)
print(firstSubstrLen + secondSubstrLen + firstLeftOver + secondLeftOver + thirdLeftOver)
print(firstSubstrLen)
print(secondSubstrLen)
print(firstLeftOver)
print(secondLeftOver)
print(thirdLeftOver)
lengths.append(firstLeftOver)
lengths.append(firstSubstrLen)
lengths.append(secondLeftOver)
lengths.append(secondSubstrLen)
lengths.append(thirdLeftOver)

firstSkipgramPart = word[: firstLeftOver]
secondSkipgramPart = word[firstLeftOver + firstSubstrLen : firstLeftOver + firstSubstrLen + secondLeftOver]
thirdSkipgramPart = word[firstLeftOver + firstSubstrLen + secondLeftOver + secondSubstrLen :]
skipgram = firstSkipgramPart + secondSkipgramPart + thirdSkipgramPart
firstRemoved = word[firstLeftOver : firstLeftOver + firstSubstrLen]
secondRemoved = word[firstLeftOver + firstSubstrLen + secondLeftOver : firstLeftOver + firstSubstrLen + secondLeftOver + secondSubstrLen]
print(firstSkipgramPart)
print(secondSkipgramPart)
print(thirdSkipgramPart)

print("Skipgram: " + skipgram)
print("First removed part: " + firstRemoved)
print("Second removed part: " + secondRemoved)

print("\n")
#Need to check the put-back mechanism as well
#If all three fit, the word is good, and we need to put it back together
restoreWord = ""
restoreWord = restoreWord + skipgram[:lengths[0]]
restoreWord = restoreWord + firstRemoved
if secondRemoved == "":
  restoreWord = restoreWord + skipgram[lengths[0]:]
else:
  restoreWord = restoreWord + skipgram[lengths[0] : lengths[0] + lengths[2]]
  restoreWord = restoreWord + secondRemoved
  restoreWord = restoreWord + skipgram[lengths[0] + lengths[2] :]
print("All three: " + restoreWord)
#Second part doesn't fit, so we put together only the first and main part
restoreWord = ""
restoreWord = restoreWord + skipgram[:lengths[0]]
restoreWord = restoreWord + firstRemoved
restoreWord = restoreWord + skipgram[lengths[0]:]
print("Main and first: " + restoreWord)
#First part doesn't fit, so we put together the main and second part
restoreWord = ""
restoreWord = restoreWord + skipgram[: lengths[0] + lengths[2]]
restoreWord = restoreWord + secondRemoved
restoreWord = restoreWord + skipgram[lengths[0] + lengths[2] :]
print("Main and second: " + restoreWord)
#Main part doesn't fit, so we put together the first and second removed parts
restoreWord = firstRemoved + secondRemoved
print("First and second: " + restoreWord)

abcdefghijklmno
15
5
1
5
1
3
abcde
k
mno
Skipgram: abcdekmno
First removed part: fghij
Second removed part: l


All three: abcdefghijklmno
Main and first: abcdefghijkmno
Main and second: abcdeklmno
First and second: fghijl


In [ ]:
#Test
word = "доярівкою"
Skipg = "дяріою"
Fst = "о"
Snd = "вк"
levenSkipgram = LevensteinSimilarity(Skipg, wordList, 15)
print("\n")
levenFstRemoved = LevensteinSimilarity(Fst, wordList, 15)
print("\n")
levenSndRemoved = LevensteinSimilarity(Snd, wordList, 15)
print(str(levenSkipgram) + " " + str(levenFstRemoved) + " " + str(levenSndRemoved))

дояркою 2
дякою 2
дірою 2
яркою 2
ярішою 2
дріадою 3
дрібкою 3
дрібною 3
дівою 3
одрою 3
різою 3
рікою 3
ріпою 3
яркішою 3
яріло 3


о 0
ов 1
од 1
ож 1
ой 1
ок 1
ом 1
он 1
ос 1
от 1
ох 1
по 1
то 1
хо 1
що 1


вак 1
вік 1
звик 2
звук 2
звяк 2
мовк 2
товк 2
увік 2
чвак 2
шовк 2
явка 2
явки 2
явко 2
явку 2
явок 2
2 0 1
